In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Duplicates

## loading data

In [ ]:
data = pd.read_excel("matlab_XLR.xlsx", sheet_name="Results Summary")
data.head()


## loading duplicates

In [ ]:
dups = pd.read_excel("matlab_XLR.xlsx", sheet_name="Duplicates")
dups.head()

In [ ]:
dups['Sample Result'] = dups['Sample Result'].str.removeprefix('<')
dups['Duplicate Result'] = dups['Duplicate Result'].str.removeprefix('<')
dups.head()

calculating range, mean, error

In [ ]:
range = np.abs(dups['Sample Result'].to_numpy('float') - dups['Duplicate Result'].to_numpy('float'))
mean = np.mean(np.c_[dups['Sample Result'].to_numpy('float'), dups['Duplicate Result'].to_numpy('float')], axis=1)
error = range / mean * 100

creating new tables

In [ ]:
nt = pd.DataFrame({'range': range, 'mean': mean, 'error (%)': error})

In [ ]:
dups = pd.concat([dups, nt], axis=1)

writing to excel ->  dups.xlsx

In [ ]:
dups.to_excel('dups.xlsx')

# liver - muscle plot

In [ ]:
data.head()

In [ ]:
vn = data.columns
for i in np.arange(3, len(vn)):
    temp = data[vn[i]].str.removeprefix("<")
    data[vn[i]] = temp


In [ ]:
data.head()

# Ploting mentioned elements

In [ ]:
# elements = ["Aluminum", "Zinc", "Mercury"]
## to plot all elements uncomment bellow
elements = data["Analyte"][1::2].to_list()

# to save the picture of an element figure, set the corrosponding value to True
# save_pic = [False, False, True]

## to save all uncomment below
save_pic = np.ones(len(elements), dtype="bool")



for i in np.arange(len(elements)):
    # creating figure
    fig, ax = plt.subplots(figsize=(15, 10))
    fig.set_facecolor("White")
    # obtaining related data
    values = np.array(data[np.logical_and(data["Analyte"] == elements[i], data["Units"] == "mg/kg wwt")].to_numpy()[:, 3:].ravel(), dtype=float)
    x = data.columns[3:]
    # seperating liver and mt using columns contatins the word "Li`ver"
    Liver_id = x.str.contains("Liver")
    # bar ploting mt
    ax.bar(x[~Liver_id], values[~Liver_id], color = "c", label="White Muscle")
    # bar ploting liver
    ax.bar(x[Liver_id], values[Liver_id], color = "k", label="Liver")
    # ploting mean of mt
    ax.plot(x[~Liver_id], np.mean(values[~Liver_id]) * np.ones(x[~Liver_id].shape), color = [0.1, 0.5, 0.8], label = "White Muscle Average", linestyle = "dashed", linewidth = 3)
    # ploting mean of liver
    ax.plot(x[Liver_id], np.mean(values[Liver_id]) * np.ones(x[Liver_id].shape), color = "gray", label = "Liver Average", linestyle = "dashed", linewidth = 3)
    # setting axis side names
    ax.set_xlabel("")
    ax.set_ylabel("mg/kg wwt")
    ax.set_title(elements[i])
    # setting xticks and xtick labels
    ax.xaxis.set_ticks(np.arange(data.shape[1]-3))
    ax.xaxis.set_ticklabels(data.columns[3:].str.removeprefix("White Sturgeon "), rotation = 40, ha = "right")
    # enabling legend
    ax.legend()
    if not os.path.exists("./pics/"):
        os.makedirs("./pics/")
    if save_pic[i] == True:
        fig.savefig("./pics/"+elements[i]+".png")